In [1]:
import json
import os
import random
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

random.seed(42)
np.random.seed(42)

In [2]:
OUTPUT_DATA_DIR = "./output_data/"

if not os.path.exists(OUTPUT_DATA_DIR):
    os.makedirs(OUTPUT_DATA_DIR)

In [3]:
train_df = pd.read_csv(OUTPUT_DATA_DIR+"interactions_training.csv")
val_df = pd.read_csv(OUTPUT_DATA_DIR+"interactions_validation.csv")
test_df = pd.read_csv(OUTPUT_DATA_DIR+"interactions_testing.csv")

full_interactions = pd.concat([train_df, val_df, test_df], axis=0)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
def load_mapping(mapping_file):
    """Loads the mapping from `mapping_file`.
    
    Parameters
    ----------
    mapping_file: str
        The name of the mapping file to import.
    
    Returns
    -------
    pd.DataFrame
        The DataFrame created from the mapping.
    
    """
    return pd.read_csv(os.path.join("mappings", "{}.csv".format(mapping_file)))

In [5]:
user_map = load_mapping("user_map")
book_map = load_mapping("book_map")

In [6]:
full_interactions['book_id'] = full_interactions['book_id'].apply(lambda x: str(x))
book_map['book_id'] = book_map['book_id'].apply(lambda x: str(x))

In [7]:
full_interactions = pd.merge(full_interactions, user_map, how="left", on=["user_id"])
full_interactions = pd.merge(full_interactions, book_map, how="left", on=["book_id"])

In [8]:
full_interactions

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at,...,year_month_added,year_month_updated,pub_date,is_translated,main_author,is_in_series,series_length,title_description,user_number,book_number
0,55e33e664f052f3858d8df5d9cd38597,2547,a3a8cf3cead1f647c240ce56d394090d,True,5,NaN,Fri Oct 18 14:39:29 -0700 2013,Fri Oct 18 14:39:29 -0700 2013,NaN,NaN,...,2013-10,2013-10,2010-01,0,6466154.0,0,1,"The Prophet Kahlil Gibran's masterpiece, The P...",36322,0
1,be813d0bb8f4cd8c4bd77870df3c8138,2696,12774740d9a76a6dc5e2f60289358cf2,True,2,NaN,Tue Feb 07 01:27:13 -0800 2012,Tue Feb 07 01:27:13 -0800 2012,NaN,NaN,...,2012-02,2012-02,2003-01,0,1838.0,0,1,The Canterbury Tales The procession that cross...,80911,1
2,695d92b05d837223608e5effcbefa14c,30119,9cc215a3037c26944369c2b839c657fc,True,4,NaN,Fri Apr 23 06:36:42 -0700 2010,Fri Apr 23 06:36:42 -0700 2010,NaN,NaN,...,2010-04,2010-04,2002-11,0,435477.0,0,1,Where the Sidewalk Ends Where the Sidewalk End...,44651,2
3,841232985badbdc753f6179ab2dcf899,26596,42b2bb4a3396fd2c63ee3fdba160fb03,False,0,NaN,Thu Sep 25 23:17:27 -0700 2014,Thu Sep 25 23:17:28 -0700 2014,NaN,NaN,...,2014-09,2014-09,1994-00,0,10547.0,0,1,"Complete Poems, 1904-1962 At the time of his d...",55994,3
4,08ff9ebbf8f94cf1d53b1fd87f077d6b,406373,0ee7eb8e9346ca6805f355fc9d8ad806,True,4,NaN,Tue Sep 18 15:41:21 -0700 2007,Wed Feb 13 07:23:38 -0800 2013,NaN,NaN,...,2007-09,2013-02,-00,0,285217.0,1,2,Faust Goethe's Faust reworks the late medieval...,3813,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326900,41c142fb5fd1e4398464ddeb7bf0e6c4,57903,fcd7aca7abecdad2665c4d06ac848f81,False,0,NaN,Tue Feb 14 01:36:58 -0800 2012,Tue Feb 14 01:36:58 -0800 2012,NaN,NaN,...,2012-02,2012-02,2001-00,0,1742.0,0,1,Pomes All Sizes The original manuscript of thi...,27700,59
326901,2a56f9db9738e3a3faa5380544960fdc,128458,d496e10809dbe3dbc82034300460eae9,False,0,NaN,Sat May 24 08:30:04 -0700 2008,Sat May 24 08:30:04 -0700 2008,NaN,NaN,...,2008-05,2008-05,2004-11,0,72605.0,0,1,I Promise to Be Good: The Letters of Arthur Ri...,17731,1304
326902,bd96ca5c84c5f4664593fb8538f6f545,118389,39c6230dbcb7e27dc38c1298b7667f88,False,0,NaN,Fri Nov 01 10:01:41 -0700 2013,Fri Nov 01 10:01:41 -0700 2013,NaN,NaN,...,2013-11,2013-11,1976-00,0,18540.0,0,1,The Love Song of J. Alfred Prufrock and Other ...,80529,405
326903,e2766e509f4a21c48ad489ffc1c3ed50,15754008,abc04b24346754a72f2f81371758dc83,True,5,NaN,Thu May 22 19:39:58 -0700 2014,Fri Apr 03 10:55:23 -0700 2015,NaN,NaN,...,2014-05,2015-04,2012-05,0,4624490.0,1,2,"The Works of Edgar Allan Poe, Volume 2 (The Wo...",95963,1931


In [9]:
def create_user_book_id(data_df):
    """Creates a user_book_id in `data_df` for joining.
    
    Parameters
    ----------
    data_df: pd.DataFrame
        The DataFrame for which the ID field is created.
    
    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from `data_df` after adding
        a user_book_id field.
    
    """
    data_df['user_number'] = data_df['user_number'].apply(lambda x: str(x))
    data_df['book_number'] = data_df['book_number'].apply(lambda x: str(x))
    data_df['user_book_id'] = data_df['user_number'] + "-" + data_df['book_number']
    return data_df.drop(columns=['user_number', 'book_number'])

In [10]:
full_interactions = create_user_book_id(full_interactions)

In [11]:
train_set = create_user_book_id(load_mapping("goodreads_train"))
val_set = create_user_book_id(load_mapping("goodreads_val"))
test_set = create_user_book_id(load_mapping("goodreads_test"))

In [12]:
train_set['is_train'] = 1
val_set['is_val'] = 1
test_set['is_test'] = 1

In [14]:
inter_train = pd.merge(full_interactions, train_set, how='left', on=['user_book_id'])
inter_train_val = pd.merge(inter_train, val_set, how='left', on=['user_book_id'])
inter_all = pd.merge(inter_train_val, test_set, how='left', on=['user_book_id'])

In [15]:
train_df = inter_all[inter_all['is_train'] == 1]
val_df = inter_all[inter_all['is_val'] == 1]
test_df = inter_all[inter_all['is_test'] == 1]

In [16]:
drop_cols = ['user_book_id', 'is_train','is_val', 'is_test']
train_df = train_df.drop(columns=drop_cols)
val_df = val_df.drop(columns=drop_cols)
test_df = test_df.drop(columns=drop_cols)

In [40]:
train_chain = train_df[['user_id', 'shelved', 'read', 'rated', 'recommended']]
train_chain_users = train_chain.groupby(train_chain['user_id'])[['shelved', 'read', 'rated', 'recommended']].sum()

In [41]:
train_chain_users['shelved_count'] = train_chain_users['shelved']
train_chain_users['read_count'] = train_chain_users['read']
train_chain_users['rated_count'] = train_chain_users['rated']
train_chain_users['recommended_count'] = train_chain_users['recommended']
train_chain_users = train_chain_users[['read_count', 'rated_count', 'recommended_count', 'shelved_count']]

In [42]:
train_chain_users = train_chain_users.reset_index()

In [43]:
train_chain_users.describe()

,read_count,rated_count,recommended_count,shelved_count
count,108389.000000,108389.000000,108389.000000,108389.000000
mean,1.583509,1.545775,1.351383,2.377040
std,2.309948,2.251070,1.851384,5.053172
min,0.000000,0.000000,0.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000
75%,2.000000,2.000000,1.000000,2.000000
max,133.000000,133.000000,133.000000,296.000000


In [44]:
def merge_user_stats(data_df, user_stats_df, is_train):
    comb_df = pd.merge(data_df, user_stats_df, how='left', on=['user_id'])
    if is_train:
        for level in ['shelved', 'read', 'rated', 'recommended']:
            count_var = "{}_count".format(level)
            comb_df[count_var] = comb_df[count_var] - comb_df[level]
    return comb_df

In [45]:
train_df_aug = merge_user_stats(train_df, train_chain_users, True)
val_df_aug = merge_user_stats(val_df, train_chain_users, False)
test_df_aug = merge_user_stats(test_df, train_chain_users, False)

In [53]:
def save_interaction_data_to_csv(interaction_df, suffix):
    """Saves the data in `interaction_df` to csv.
    
    Parameters
    ----------
    interaction_df: pd.DataFrame
        The DataFrame being saved to csv.
    suffix: str
        A suffix added to the saved file to identify the
        particular interactions data.
    
    Returns
    -------
    None
    
    """
    file_path = "{0}interactions_{1}.csv".format(OUTPUT_DATA_DIR, suffix)
    interaction_df.to_csv(file_path, index=False)

In [54]:
save_interaction_data_to_csv(train_df_aug, "training")
save_interaction_data_to_csv(test_df_aug, "testing")
save_interaction_data_to_csv(val_df_aug, "validation")